In [1]:
import os
import itertools
import glob
import datetime
import numpy as np
import pandas as pd
from datetime import *

In [24]:
root_dir = os.getcwd()
conf_dir = 'conferences'
data_dir = os.path.join(root_dir, "data")


def load_csvs(file_names):
    """Loads and concatentates csv's from a directory"""
    df = pd.DataFrame()
    for each_file in file_names:
        new_df = pd.read_csv(each_file)
        df = pd.concat([df, new_df])
    return df

def join_data(scores_df, stats_df, odds_df):
    """
    Creates a unique key for each game using the date the game was played
    and the home and away abbreviated names (Not all data sets have a HomeID
    and AwayID)
    """

    # Add dates to join on
    scores_df['Year'] = scores_df['WeekDate'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").year)
    scores_df['Month'] = scores_df['WeekDate'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").month)
    scores_df['Day'] = scores_df['WeekDate'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").day)
    stats_df['Year'] = stats_df['Start'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").year)
    stats_df['Month'] = stats_df['Start'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").month)
    stats_df['Day'] = stats_df['Start'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").day)
    odds_df['Year'] = odds_df['DATE(date)'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").year)
    odds_df['Month'] = odds_df['DATE(date)'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").month)
    odds_df['Day'] = odds_df['DATE(date)'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").day)

    # Join Data
    data = scores_df.merge(
        stats_df.drop(['Season', 'Start', 'Week'], axis=1),
        left_on = ['Year', 'Month', 'Day', 'Home', 'Visiter'],
        right_on = ['Year', 'Month', 'Day', 'Home', 'Away'])
    data = data.merge(odds_df.drop(['DATE(date)', 'HomeScore', 'AwayScore'],
                                    axis=1),
        left_on = ['Year', 'Month', 'Day', 'Home', 'Visiter'],
        right_on = ['Year', 'Month', 'Day', 'Home', 'Away'])

    # Target feature
    data['target_margin'] = data['HomeFinal'] - data['VisFinal']

    # Other features
    data['D1_Match'] = [True if not pd.isnull(x) else False for \
                        x in data['Spread_Mirage']]

    return data

# Load data locations
scores_dir = 'scores_pe'
stats_dir = 'stats'
odds_dir = ''

scores_names = glob.glob(os.path.join(root_dir, data_dir, scores_dir, "scores_pythElo201?.csv"))
stats_names =  glob.glob(os.path.join(root_dir, data_dir, stats_dir, "ncaastats201?.csv"))
odds_names = [os.path.join(root_dir, data_dir, odds_dir, "NCAAF_Odds.csv")]

# Import data and join
scores_df = load_csvs(scores_names)
stats_df = load_csvs(stats_names)
odds_df = load_csvs(odds_names)
data = join_data(scores_df, stats_df, odds_df)

spreads = data.set_index(['HomeID','VisID','Season','Week']).filter(regex="Spread_")
#m = spreads.mean(axis=1)
#for i, col in enumerate(spreads):
    # using i allows for duplicate columns
    # inplace *may* not always work here, so IMO the next line is preferred
    # df.iloc[:, i].fillna(m, inplace=True)
    #spreads.iloc[:, i] = spreads.iloc[:, i].fillna(m)
# spreads['target_margin'] = data['target_margin']
#spreads.dropna(axis=0, inplace=True)
spreads = spreads.join(pd.DataFrame(data.set_index(['HomeID','VisID','Season','Week'])['target_margin']))

# Join Conference Data
file = os.path.join(data_dir, conf_dir, "mergedConferences.csv")
conf_df = pd.read_csv(file).drop_duplicates()
spreads= spreads.reset_index().merge(conf_df,
                                            left_on=['HomeID', 'Season'],
                                            right_on=['ID','Year'],
                                            suffixes=('','Home'))
spreads = spreads.reset_index().merge(conf_df,
                                            left_on=['VisID', 'Season'],
                                            right_on=['ID','Year'],
                                            suffixes=('','Vis'))
spreads['Week'] = spreads['Week'].astype(int)
spreads['Week'] = np.where(spreads['Season']==2016, spreads['Week'] - 1, spreads['Week'])
spreads = spreads.set_index(['HomeID', 'VisID', 'Season', 'Week'])
spreads = spreads.drop(['ID','Year','IDVis','index','Team','TeamVis','ConfVis','Year','YearVis'],1)

# SPLIT FOR USE IN R

In [140]:

for y, g in spreads2.groupby('Season'):
    weeks = [group for _, group in g.groupby('Week')]
    for i, w in enumerate(weeks):
        i += 1
        if i == 4:
            pd.concat(weeks[:i]).to_csv('data/new_odds/pre/pre_{}/odds{}_{}.csv'.format(y, y, i), index=False)
        elif i>4:
            w.to_csv('data/new_odds/pre/pre_{}/odds{}_{}.csv'.format(y, y, i), index=False)

# RECOMBINE AFTER R

In [ ]:
new_oddsList = [0 for i in range(4)]

for i, yr in enumerate(range(2013,2017)):
    lst_wk = spreads2.loc[spreads2['Season']==yr,'Week'].max()
    new_oddsList[i] = pd.read_csv('data/new_odds/post/post_{}/odds{}_{}.csv'.format(yr, yr, lst_wk))
    
new_spreads = pd.concat(new_oddsList)
new_spreads['Spread_Med2'] = new_spreads[casinos].median(axis=1)
new_spreads['Spread_Mode2'] = new_spreads[casinos].mode(axis=1)[0]
new_spreads.to_csv('data/new_odds/new_odds.csv', index=False)

# MODELING

In [217]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge

In [ ]:
orig_casinos = ['Spread_Mirage', 'Spread_Pinnacle', 'Spread_Sportsbet', 
               'Spread_Westgate', 'Spread_Station', 'Spread_SIA',
               'Spread_SBG', 'Spread_BetUS']
orig_med = ['Spread_Med']
orig_mode = ['Spread_Mode']
orig_summaries = ['Spread_Med', 'Spread_Mode']
orig_total = ['Spread_Mirage', 'Spread_Pinnacle', 'Spread_Sportsbet', 
               'Spread_Westgate', 'Spread_Station', 'Spread_SIA',
               'Spread_SBG', 'Spread_BetUS', 'Spread_Med', 'Spread_Mode']

new_med = ['Spread_Med2']
new_mode = ['Spread_Mode2']
new_summaries = ['Spread_Med2', 'Spread_Mode2']
new_total = ['Spread_Mirage', 'Spread_Pinnacle', 'Spread_Sportsbet', 
               'Spread_Westgate', 'Spread_Station', 'Spread_SIA',
               'Spread_SBG', 'Spread_BetUS', 'Spread_Med', 'Spread_Mode']
new_total_total = ['Spread_Mirage', 'Spread_Pinnacle', 'Spread_Sportsbet', 
               'Spread_Westgate', 'Spread_Station', 'Spread_SIA',
               'Spread_SBG', 'Spread_BetUS', 'Spread_Med', 'Spread_Mode',
                'Spread_Med2', 'Spread_Mode2']

features = [('orig_casinos', orig_casinos), 
            ('orig_med', orig_med), ('orig_mode', orig_mode), ('orig_summaries', orig_summaries), ('orig_total', orig_total),
            ('new_med', new_med), ('new_mode', new_mode), ('new_summaries', new_summaries), ('new_total', new_total)]


In [228]:
for name, feats in features:
    new_feats = feats.copy()
    new_feats.append('Season')
    new_feats.append('target_margin')
    new_spreadTemp = new_spreads[new_feats].copy().dropna()
    
    X_train = new_spreadTemp.loc[new_spreadTemp['Season']<2016,feats]
    X_val = new_spreadTemp.loc[new_spreadTemp['Season']==2016,feats]
    y_train = new_spreadTemp.loc[new_spreadTemp['Season']<2016,'target_margin']
    y_val = new_spreadTemp.loc[new_spreadTemp['Season']==2016,'target_margin']
    
    standardscaler = StandardScaler()
    X_trainS = standardscaler.fit_transform(X_train)
    X_valS = standardscaler.transform(X_val)
    
    print('{}: {}' .format(name, mean_squared_error(y_val, BayesianRidge().fit(X_trainS, y_train).predict(X_valS))))

orig_casinos: 261.4307500676896
orig_med: 272.0562382987377
orig_mode: 272.2678668409124
orig_summaries: 272.2888260225782
orig_total: 260.2619124221732
new_med: 273.6773607758476
new_mode: 274.2999240907326
new_summaries: 273.496634545673
new_total: 260.2619124221732


In [153]:
for name, feats in [('orig_casinos',orig_casinos), ('orig_total',orig_total)]:
    new_feats = feats.copy()
    new_feats.append('Season')
    new_feats.append('target_margin')
    new_spreadTemp = new_spreads[new_feats].copy().dropna()
    
    X_train = new_spreadTemp.loc[new_spreadTemp['Season']<2016,feats]
    X_val = new_spreadTemp.loc[new_spreadTemp['Season']==2016,feats]
    y_train = new_spreadTemp.loc[new_spreadTemp['Season']<2016,'target_margin']
    y_val = new_spreadTemp.loc[new_spreadTemp['Season']==2016,'target_margin']
    
    standardscaler = StandardScaler()
    X_trainS = standardscaler.fit_transform(X_train)
    X_valS = standardscaler.transform(X_val)
    
    X_train_val = np.vstack((X_trainS, X_valS))
    y_train_val = np.concatenate((y_train, y_val))
    val_fold = [-1]*len(X_trainS) + [0]*len(X_valS) #0 corresponds to validation
    grid = GridSearchCV(BayesianRidge(),
                        [{'alpha_1':10**np.arange(0, 3, 0.25),
                          'alpha_2':10**np.arange(-10, -7, 0.25),
                          'lambda_1':10**np.arange(-10, -7, 0.25),
                          'lambda_2':10**np.arange(0, 3, 0.25)}],
                        return_train_score=False,
                        cv = PredefinedSplit(test_fold=val_fold),
                        refit = True,
                        scoring = make_scorer(mean_squared_error, greater_is_better = False))
    grid.fit(X_train_val, y_train_val)
    
    bestimator = grid.best_estimator_
    print(name)
    print(grid.best_params_ )
    print('MSE: {}' .format(mean_squared_error(y_val, bestimator.fit(X_trainS, y_train).predict(X_valS))))
    print('*****************************************************')

orig_casinos
{'alpha_1': 1.0, 'alpha_2': 5.6234132519034905e-08, 'lambda_1': 5.6234132519034905e-08, 'lambda_2': 1.0}
MSE: 261.46309534670485
*****************************************************
orig_total
{'alpha_1': 1.0, 'alpha_2': 5.6234132519034905e-08, 'lambda_1': 5.6234132519034905e-08, 'lambda_2': 1.0}
MSE: 260.3163109699554
*****************************************************


In [225]:
for name, feats in [('orig_casinos',orig_casinos), ('orig_total',orig_total)]:
    new_feats = feats.copy()
    new_feats.append('Season')
    new_feats.append('target_margin')
    new_spreadTemp = new_spreads[new_feats].copy().dropna()
    
    X_train = new_spreadTemp.loc[new_spreadTemp['Season']<2016,feats]
    X_val = new_spreadTemp.loc[new_spreadTemp['Season']==2016,feats]
    y_train = new_spreadTemp.loc[new_spreadTemp['Season']<2016,'target_margin']
    y_val = new_spreadTemp.loc[new_spreadTemp['Season']==2016,'target_margin']
    
    standardscaler = StandardScaler()
    X_trainS = standardscaler.fit_transform(X_train)
    X_valS = standardscaler.transform(X_val)
    
    X_train_val = np.vstack((X_trainS, X_valS))
    y_train_val = np.concatenate((y_train, y_val))
    val_fold = [-1]*len(X_trainS) + [0]*len(X_valS) #0 corresponds to validation
    grid = GridSearchCV(SGDRegressor(loss='epsilon_insensitive', penalty='l1', max_iter=10000),
                        [{'alpha':10**np.arange(-10,1,1.0),
                          'eta0':10**np.arange(-5,2,1.0),
                          'learning_rate':['constant','optimal','invscaling']}],
                        return_train_score=False,
                        cv = PredefinedSplit(test_fold=val_fold),
                        refit = True,
                        scoring = make_scorer(mean_squared_error, greater_is_better = False))
    grid.fit(X_train_val, y_train_val)
    
    bestimator = grid.best_estimator_
    print(name)
    print(grid.best_params_ )
    print('MSE: {}' .format(mean_squared_error(y_val, bestimator.fit(X_trainS, y_train).predict(X_valS))))
    print('*****************************************************')

orig_casinos
{'alpha': 1e-09, 'eta0': 0.0001, 'learning_rate': 'invscaling'}
MSE: 256.8414429758759
*****************************************************
orig_total
{'alpha': 0.001, 'eta0': 0.0001, 'learning_rate': 'invscaling'}
MSE: 255.35641933873944
*****************************************************


In [227]:
bestimator.score(X_valS, y_val)

0.26430062120519215